In [1]:
# Interact with the UI on Cell 3

# Install required packages in the notebook
!pip3 install --quiet openai requests python-dotenv ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter labextension install @jupyter-widgets/jupyterlab-manager




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` no

In [2]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML


# Load environment variables and set OpenAI API key
load_dotenv()
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)


In [23]:
# UI components
button = widgets.Button(description="Click Me for a Joke",button_style='danger', layout=widgets.Layout(width="400px", height="130px"))
button.style.button_color = '#4C69E1'
button.style.font_size = "2rem"

output_area = widgets.Output()

style = """
<style>
    .joke-container {
        font-family: Arial, sans-serif;
        border-radius: 12px;
        max-width: 20rem;
        padding: 1.5rem;
        margin: 15px 0;
        background-color: #f9fafb;
        border: 1px solid #e5e7eb;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    
    .joke-container:first-child {
        background-color: #e0f2fe;
        border-color: #93c5fd;
        max-width: 30rem;
        box-shadow: 0 4px 10px rgba(37, 99, 235, 0.3);
    }

    .joke-container:first-child .joke-content {
        font-size: 2.25remrem;
        font-weight: bold;
    }
    
    .joke-style {
        font-size: 0.75rem;
        letter-spacing: 0.05em;
        text-transform: uppercase;
        color: #374151;
        margin-bottom: 1rem;
    }

    .joke-content {
        color: #111827;
        font-weight: normal;
        margin-bottom: 0.5rem;
        font-size: 1.5rem;
        font-weight: semibold;
    }
    .html-container {
        display: flex;
        flex-direction: column;
        align-items: center;
        font-family: Arial, sans-serif;
        color: #1f2937;
    }

</style>
"""

def on_button_click(b):
    button.layout.display = 'none'
    with output_area:
        clear_output()  # Clear previous output
        display(HTML(jokesStyles))  # Display CSS styles

        joke_variations = get_joke_variations()
        
        jokes_html = '<div class="html-container">'
        for joke in joke_variations:
            jokes_html += f"""
                <div class="joke-container">
                    <div class="joke-style">{joke['style']} Style</div>
                    <div class="joke-content">{joke['joke']}</div>
                </div>
            """
        jokes_html += '</div>'
        display(HTML(jokes_html))

# Set the button to call `on_button_click` when clicked
button.on_click(on_button_click)

# Display the UI
display(HTML(style))  
display(button)
display(output_area)



Button(button_style='danger', description='Click Me for a Joke', layout=Layout(height='130px', width='400px'),…

Output()

In [4]:
def get_joke_data():
    url = "https://v2.jokeapi.dev/joke/Any?blacklistFlags=political,racist,sexist"
    response = requests.get(url)
    if response.status_code == 200:
        joke_data = response.json()
        if joke_data.get("type") == "single":
            joke_text = joke_data["joke"]
        else:
            joke_text = f"{joke_data['setup']} ... {joke_data['delivery']}"
        return joke_text
    else:
        print("Error fetching joke data.")
        return None
    

In [5]:
def generate_variation(joke_text, style):
    prompt = f"Rewrite the following joke in {style} style: {joke_text}"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50,
        temperature=0.7,
    )
    return response.choices[0].message.content.strip()

In [6]:
def get_joke_variations():
    joke_text = get_joke_data()
    if not joke_text:
        return "No joke available."

    styles = ["Original", "NSFW", "Gen Alpha", "Gen Z", "Millennial", "Boomer"]
    joke_variations = []

    for style in styles:
        variation = generate_variation(joke_text, style)
        joke_variations.append({"style": style, "joke": variation})

    return joke_variations